In [1]:
import math
import ROOT
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.24/06


In [34]:
Dir = '../data/'
    # directory that stores input data
    
# dictionary storing the pairs of input-output files.
files = {"rock_g_K40.root"   : "rockGammaK40.root" ,\
         "rock_g_U238.root"  : "rockGammaU238.root",\
         "rock_g_Th232.root" : "rockGammaTh232.root",\
         "rock_n_Unat.root"  : "rockNeutronUnat.root",\
         "rock_n_Th232.root" : "rockNeutronTh232"
        }
    
# dict that stores total number of simulations, used to deduce simulation time
Nparticle = {"rock_g_K40.root"   : 1000000000 * 10,\
             "rock_g_U238.root"  : 50000000 * 25,\
             "rock_g_Th232.root" : 50000000 * 25,\
             "rock_n_Unat.root"  : 4000000 * 25,\
             "rock_n_Th232.root" : 4000000 * 25
            }
    
# set some constants:
config = { 'density' : 3.26,\
              # g/cm3
          'radius' : 100,\
              # cm
          'halfz' : 25\
              # cm
         }
config['volume'] = math.pi*config['radius']**2*config['halfz']*2 # cm3
config['mass'] = config['volume'] * config['density'] / 1000 # total mass in kg
config['activity'] = 1     # target activity in Bq/kg
config['rate'] = config['mass'] * config['activity']    # in Hz

for Input,Output in files.items():
    print('Procecssing', Input)

    # Get the cos(theta) vs E
    # first prepare some expressions for ROOT plotting /  generating histograms
    expr = "pz/sqrt((px*px)+(py*py)+(pz*pz)):Eki"
        # ROOT expression to be plotted.
    histname = 'theta_vs_E'
        # Name of histogram in output file
    hist = "(3000,500,3000,1000,0,1)"
        # Historam to store the result
    cut = "pz>0 && particle==\"gamma\" && Eki>500"
        # selection criteria

    # if the file is for neutron, then change energy range and particle type.
    if "Neutron" in Output:
        print('Neutron simulation, setting range and cuts.')
        hist = "(1000,0,10000,1000,0,1)"
        cut = "pz>0 && particle==\"neutron\""
        
        if "Unat" in Output:
            # Neutron rate from 1 ppb of U-238 in rock with 1 g/cm3-density
            config['n_rate'] = 4.914E-11            # neutrons / sec / cm3, since density is 1 g/cm3, it's also n/s/g
            # U238: 1 ppb is 12.3 mBq/kg, so 10 ppb is 123 mBq/kg, or 0.123 Bq/kg
            config['rate'] = config['n_rate'] * (config['density'] * config['volume']) / 0.0123
        if "Th232" in Output:
            config['n_rate'] = 1.685E-11
            config['rate'] = config['n_rate'] * config['density'] * config['volume'] / 0.0041

    file1 = ROOT.TFile( Dir+Input, "READ")
    tree1 = file1.Get("events")
    
    ROOT.gROOT.cd()
    
    # Generate the 2D histogram to be written in output using all data.
    tree1.Draw( expr+'>>'+histname+hist, cut )
    thetaE1_pdf = ROOT.gROOT.Get(histname)
    thetaE1_pdf.Scale( 1./thetaE1_pdf.GetEntries() )
    
    config['Ntotal'] = Nparticle[Input]
    config['duration'] = config['Ntotal']/config['rate']
          # in second
    
    config['r_probe'] = 50 # in cm
    cut2 = cut + " && sqrt((rx*rx)+(ry*ry))<{}".format(config['r_probe']*10)
    config['Ndetected'] = tree1.Draw( expr+'>>'+histname+'_flux'+hist, cut2 )
    config['flux'] = config['Ndetected']/( config['duration'] * math.pi * config['r_probe']**2 )
          
    # write them to output
    file2 = ROOT.TFile( Dir+Output, "RECREATE")
    thetaE1_pdf.Write()
    
    note = ROOT.TMacro('config')
    for key in config:
        note.AddLine( key+' {}'.format(config[key]) )
    note.Write()
    
    file2.Close()
    print(Output, 'written.' )
    print('Flux per Bq/kg is {:.5e}'.format(config['flux']), '/cm2/s' )

Procecssing rock_g_K40.root
rockGammaK40.root written.
Flux per Bq/kg is 9.10976e-04 /cm2/s
Procecssing rock_g_U238.root
rockGammaU238.root written.
Flux per Bq/kg is 8.93298e-03 /cm2/s
Procecssing rock_g_Th232.root
rockGammaTh232.root written.
Flux per Bq/kg is 1.17790e-02 /cm2/s
Procecssing rock_n_Unat.root
Neutron simulation, setting range and cuts.
rockNeutronUnat.root written.
Flux per Bq/kg is 9.68031e-08 /cm2/s
Procecssing rock_n_Th232.root
Neutron simulation, setting range and cuts.
rockNeutronTh232 written.
Flux per Bq/kg is 1.00497e-07 /cm2/s
